
## The Battle of Neighborhoods

###  Course: Applied Data Science Capstone  


This notebook would contain Explaination and code for the coursera Capstone Project.

### Adding Required Libraries to create required Environment

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.expand_frame_repr', False)

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## 1. Data scraping

#### Description: 
Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [4]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
nbr = pd.DataFrame(columns=column_names)
nbr

,Postal_Code,Borough,Neighborhood


#### Web scraping

##### Importing Data From provided URL

In [5]:
#import the library used to query a website
from urllib.request import urlopen

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the wiki to the variable 'page'
page = urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")

In [14]:
# Uncomment below to Print HTML of the Page
#print(soup.prettify()) 

#Get table 
right_table=soup.find('table', class_='wikitable sortable')

right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

##### Extract the information to DataFrame: 

we iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list.

In [7]:
#Generate lists
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
nbr['Postal_Code']=A
nbr['Borough']=B
nbr['Neighborhood']=C

#uncomment below to view dataframe nbr which contains data as retreived from the webpage in wiki
#nbr


####  Ignore cells with a borough that is Not assigned.

1. Compare with Each row if Borough is Not Assigned drop row.
2. Reset Index.

In [8]:
nbr = nbr.drop(nbr[nbr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
nbr.index = pd.RangeIndex(len(nbr.index))


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

instantiate additional dataframe nbrNA.
Compare with Each row if Neighborhood is Not Assigned add value from Borough.

In [9]:
# instantiate additional dataframe nbrNA
column_n = ['NBRVAL'] 
nbrNA = pd.DataFrame(columns=column_n)
D=['Not assigned'] 
nbrNA['NBRVAL'] = D

#Dataframe contains 'Not assigned' value used for Comparison.
#print(nbrNA)

nbr1=nbr
#nbr1


#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index,row in nbr.iterrows():
    if((nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
        #print('inside if')
        nbr1.loc[row_index,['Neighborhood']] = nbr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe nbr1 which contains Neighborhoods which were initially 'Not Assigned' now containing Borough values
print('nbr1')
nbr1

nbr1


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


##### More than one neighborhood can exist in one postal code area.

In [10]:
nbr2=nbr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

nbr2

,Postal_Code,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood\n, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae\n,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park\n, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West\n",Scarborough
9,M1N,"Birch Cliff, Cliffside West\n",Scarborough


In [11]:
print(nbr2.shape)
print('nbr2')
nbr2


(103, 3)
nbr2


,Postal_Code,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood\n, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae\n,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park\n, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West\n",Scarborough
9,M1N,"Birch Cliff, Cliffside West\n",Scarborough
